<header> Mimic III v1.4 Investigation </header> 

### Part 1: Import the data

All data has been downloaded from the Mimic-III website:

MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available from: http://www.nature.com/articles/sdata201635

Access can be granted via PhysioNetWorks:

Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220 [Circulation Electronic Pages; http://circ.ahajournals.org/content/101/23/e215.full]; 2000 (June 13).

First we need to import the needed modules.  Let us test psycopg2 which will allow us to access our PSQL database using Python.

In [1]:
#Import necessary modules

import psycopg2
import psycopg2.extras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score


#Connect to PSQL database
try:
    conn = psycopg2.connect(dbname="mimic", user="postgres", password="Postgres89")
except:
    print "I am unable to connect to the database"

#Create cursor to access tables from database    
curr = conn.cursor()
try:
    curr.execute("SELECT LOS from ICUSTAYS")
except:
    print "I cannot SELECT from ADMISSIONS"

#Fetch all rows from table    
rows = curr.fetchall()
print rows[1]
rowcount = curr.rowcount
print "Length is ", rowcount
print "\nRows: \n"

#Example creating groups of patients based on length of stay (LOS)
none_count =0
los_sum = 0.
los_groups = {"<1 day": 0, "1-2 days": 0, "2-3 days": 0, "3-5 days": 0, "5-10 days":0, "10+ days": 0}
for record in rows:
    if record[0] is not None:
        los_sum = los_sum + record[0]
        if record[0] <= 1.0:
            los_groups["<1 day"] = los_groups["<1 day"]+1
        elif record[0]<=2.0:
            los_groups["1-2 days"] = los_groups["1-2 days"]+1
        elif record[0]<=3.0:
            los_groups["2-3 days"] = los_groups["2-3 days"]+1
        elif record[0]<=5.0:
            los_groups["3-5 days"] = los_groups["3-5 days"]+1
        elif record[0]<=10.0:
            los_groups["5-10 days"] = los_groups["5-10 days"]+1
        else:
            los_groups["10+ days"] = los_groups["10+ days"]+1
        
    else:
        none_count += 1
        
        
los_average = los_sum/(rowcount-none_count)
print "Number of rows with no LOS data", none_count
print "Average Length of Stay is ", los_average
print los_groups

#Close cursor after finished accessing table
curr.close()

(3.2788,)
Length is  61532

Rows: 

Number of rows with no LOS data 10
Average Length of Stay is  4.9179715809
{'10+ days': 6771, '2-3 days': 9644, '<1 day': 12311, '3-5 days': 8801, '1-2 days': 16901, '5-10 days': 7094}


In [2]:
#Access relevant fields from ADMISSIONS table
curr2 = conn.cursor()
try:
    curr2.execute("SELECT SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME,INSURANCE, LANGUAGE,"
                  "RELIGION, MARITAL_STATUS, ETHNICITY from ADMISSIONS")
except:
    print "I cannot SELECT from ADMISSIONS"

#Get column names for Pandas DataFrame
admcolnames = [desc[0] for desc in curr2.description]    
#Create Pandas DataFrame
dfadm = pd.DataFrame(curr2.fetchall(), columns=admcolnames)
print dfadm.head() #Get data sample
print len(dfadm.index)
curr2.close()

   subject_id  hadm_id           admittime           dischtime deathtime  \
0          22   165315 2196-04-09 12:26:00 2196-04-10 15:54:00      None   
1          23   152223 2153-09-03 07:15:00 2153-09-08 19:10:00      None   
2          23   124321 2157-10-18 19:34:00 2157-10-25 14:00:00      None   
3          24   161859 2139-06-06 16:14:00 2139-06-09 12:48:00      None   
4          25   129635 2160-11-02 02:06:00 2160-11-05 14:55:00      None   

  insurance language           religion marital_status ethnicity  
0   Private     None       UNOBTAINABLE        MARRIED     WHITE  
1  Medicare     None           CATHOLIC        MARRIED     WHITE  
2  Medicare     ENGL           CATHOLIC        MARRIED     WHITE  
3   Private     None  PROTESTANT QUAKER         SINGLE     WHITE  
4   Private     None       UNOBTAINABLE        MARRIED     WHITE  
58976


In [3]:
#Access relevant fields from PATIENTS table
curr3 = conn.cursor()
try:
    curr3.execute("SELECT SUBJECT_ID, GENDER, DOB, DOD, EXPIRE_FLAG from PATIENTS")
except:
    print "I cannot SELECT from PATIENTS"

#Create PATIENTS DataFrame
patcolnames = [desc[0] for desc in curr3.description]
dfpat = pd.DataFrame(curr3.fetchall(), columns=patcolnames)
print dfpat.head()
print len(dfpat.index)
curr3.close()
#print dfpat['expire_flag'].value_counts(dropna = False)

   subject_id gender        dob        dod  expire_flag
0         249      F 2075-03-13        NaT            0
1         250      F 2164-12-27 2188-11-22            1
2         251      M 2090-03-15        NaT            0
3         252      M 2078-03-06        NaT            0
4         253      F 2089-11-26        NaT            0
46520


#### Merge the imported data

In [4]:
#Merge ADMISSIONS and PATIENTS table
dfadmpat = dfpat.merge(dfadm, how='inner', on="subject_id")
print dfadmpat.head(n=5)
print len(dfadmpat.index)
#print dfadmpat['expire_flag'].value_counts(dropna = False)
#dfadmpat.drop_duplicates('subject_id', keep='last', inplace=True)
#print dfadmpat['expire_flag'].value_counts(dropna = False)

   subject_id gender        dob        dod  expire_flag  hadm_id  \
0         249      F 2075-03-13        NaT            0   116935   
1         249      F 2075-03-13        NaT            0   149546   
2         249      F 2075-03-13        NaT            0   158975   
3         250      F 2164-12-27 2188-11-22            1   124271   
4         251      M 2090-03-15        NaT            0   117937   

            admittime           dischtime            deathtime insurance  \
0 2149-12-17 20:41:00 2149-12-31 14:55:00                 None  Medicare   
1 2155-02-03 20:16:00 2155-02-14 11:15:00                 None  Medicare   
2 2156-04-27 15:33:00 2156-05-14 15:30:00                 None  Medicare   
3 2188-11-12 09:22:00 2188-11-22 12:00:00  2188-11-22 12:00:00  Self Pay   
4 2110-07-27 06:46:00 2110-07-29 15:23:00                 None   Private   

  language       religion marital_status               ethnicity  
0     None       CATHOLIC       DIVORCED                   WHITE  


In [5]:
from IPython.display import display

#Access relevant fields from ICUSTAYS table
curr4 = conn.cursor()
try:
    curr4.execute("SELECT SUBJECT_ID, HADM_ID, LOS from ICUSTAYS")
except:
    print "I cannot SELECT from ICUSTAYS"

#Create ICUSTAYS DataFrame    
icucolnames = [desc[0] for desc in curr4.description]
dficu = pd.DataFrame(curr4.fetchall(), columns=icucolnames)
print len(dficu.index)
print dficu.head()

dficu['los_scaled'] = np.log(dficu.los)

print max(dficu['los'])
print max(dficu['los_scaled'])

#Look for scaled LOS outliers
Q1 = np.nanpercentile(dficu['los_scaled'], 25)
print 'Q1 is', Q1
    
Q3 = np.nanpercentile(dficu['los_scaled'], 75)
print 'Q3 is', Q3

step = 5.0 * (Q3-Q1)
print "Data points considered outliers for the feature '{}':".format('los_scaled')
display(dficu['los_scaled'][~((dficu['los_scaled'] >= Q1 - step) & (dficu['los_scaled'] <= Q3 + step))])

#Merge 
dfmimic = dfadmpat.merge(dficu, how="inner", on=['hadm_id', 'subject_id'])
print len(dfmimic.index)
print dfmimic['expire_flag'].value_counts(dropna = False)
#Drop duplicate data, it can be assumed if there are multiple admissions
#of the same patient, they will have survived the earlier admissions.
dfmimic.drop_duplicates('hadm_id', inplace=True) #Drop duplicate hospital admission data
print len(dfmimic.index)
print dfmimic['expire_flag'].value_counts(dropna = False)
dfmimic.drop_duplicates('subject_id', keep='last', inplace=True) #Drop duplicate subject data
print len(dfmimic.index)
print dfmimic['expire_flag'].value_counts(dropna = False)

#Name the final DataFrame
dfmimic.name = 'dfmimic'

curr4.close()

61532
   subject_id  hadm_id     los
0         268   110404  3.2490
1         269   106296  3.2788
2         270   188028  2.8939
3         271   173727  2.0600
4         272   164716  1.6202
173.0725
5.15371058185
Q1 is 0.102579150484
Q3 is 1.50033150072
Data points considered outliers for the feature 'los_scaled':


1033          NaN
5918    -8.111728
6451          NaN
10165         NaN
10507         NaN
10771   -8.111728
13080         NaN
13346   -9.210340
15156         NaN
17759         NaN
18442         NaN
24081   -8.517193
24781   -6.907755
27840         NaN
28121   -7.130899
29117         NaN
33110   -7.418581
45725   -7.824046
48101   -7.824046
Name: los_scaled, dtype: float64

61532
0    37341
1    24191
Name: expire_flag, dtype: int64
57786
0    35744
1    22042
Name: expire_flag, dtype: int64
46476
0    30739
1    15737
Name: expire_flag, dtype: int64


Let's get counts by gender ('M' or 'F') and whether they lived or died.

In [6]:
print len(dfmimic.index)

#Group by gender
mimic_by_gender = dfmimic.groupby('gender') #DataFrame grouped by gender
print mimic_by_gender.head()
print mimic_by_gender['expire_flag'].sum() #Sum of deaths
print mimic_by_gender['expire_flag'].value_counts() #Counts of deaths vs no deaths by gender
print dfmimic['expire_flag'].sum()
print dfmimic['expire_flag'].value_counts(dropna = False)

46476
    subject_id gender        dob        dod  expire_flag  hadm_id  \
3          249      F 2075-03-13        NaT            0   158975   
5          250      F 2164-12-27 2188-11-22            1   124271   
6          251      M 2090-03-15        NaT            0   117937   
8          252      M 2078-03-06        NaT            0   193470   
9          253      F 2089-11-26        NaT            0   176189   
10         255      M 2109-08-05        NaT            0   112013   
14         256      M 2086-07-31        NaT            0   108811   
16         257      F 2031-04-03 2121-07-08            1   179006   
17         258      F 2124-09-19        NaT            0   189406   
20         261      M 2025-08-04 2102-06-29            1   118523   

             admittime           dischtime            deathtime insurance  \
3  2156-04-27 15:33:00 2156-05-14 15:30:00                 None  Medicare   
5  2188-11-12 09:22:00 2188-11-22 12:00:00  2188-11-22 12:00:00  Self Pay   
6  

We can see that our table contains data for 30739 patients that survived their stay in the ICU and 15737 patients who died in the ICU.
Let's create a benchmark model for the data that will guess all patients survived.  We will use scikit-learn's AUC function to compute the area under the curve.  

In [7]:
#Create NumPy array predicted all '1's for patient survival
mimic_length = len(dfmimic.index)
y_scores = np.ones(mimic_length, dtype=np.int)
y_true = dfmimic['expire_flag']
print roc_auc_score(y_true, y_scores)
print roc_auc_score(y_true, y_true)

0.5
1.0


Now we can find the patient's age on admission by subtracting the patient's date of birth ('dob') from the hospital's admission date ('admittime').  Patient's will be grouped by aging similarly to how the CDC grouped patient's in the National Hospital Discharge Survey 2000-2010: https://www.cdc.gov/nchs/data/databriefs/db118.htm#x2013;2010%3C/a%3E%20

In [8]:
from datetime import timedelta

def groupage(admittime, dob):
    '''
    inputs:
        -admittime: A datetime value of when the patient was admitted to the ICU
        -dob: A datetime value of the patient's Date of Birth
    output:
        The group a patient belongs to based on the difference between admittime and dob
    '''
    age_in_years = (admittime-dob)
    #Group ages based on CDC grouping
    if age_in_years >= timedelta(85*365.242):
        return '85 years and over'
    elif age_in_years >= timedelta(75*365.242):
        return '75-84 years'
    elif age_in_years >= timedelta(65*365.242):
        return '65-74 years'
    elif age_in_years >= timedelta(45*365.242):
        return '45-64 years'
    else:
        return 'Under 45 years'
    

dfmimic['age'] = dfmimic.apply(lambda row: groupage(row['admittime'], row['dob']), axis=1)
print dfmimic['age'].value_counts()
print dfmimic.head()    
    
    

Under 45 years       13620
45-64 years          12833
75-84 years           8080
65-74 years           7678
85 years and over     4265
Name: age, dtype: int64
   subject_id gender        dob        dod  expire_flag  hadm_id  \
3         249      F 2075-03-13        NaT            0   158975   
5         250      F 2164-12-27 2188-11-22            1   124271   
6         251      M 2090-03-15        NaT            0   117937   
8         252      M 2078-03-06        NaT            0   193470   
9         253      F 2089-11-26        NaT            0   176189   

            admittime           dischtime            deathtime insurance  \
3 2156-04-27 15:33:00 2156-05-14 15:30:00                 None  Medicare   
5 2188-11-12 09:22:00 2188-11-22 12:00:00  2188-11-22 12:00:00  Self Pay   
6 2110-07-27 06:46:00 2110-07-29 15:23:00                 None   Private   
8 2133-08-15 04:23:00 2133-08-19 17:30:00                 None   Private   
9 2174-01-21 20:58:00 2174-01-26 16:15:00           

Just as patient's were grouped into different age groups, patients will now be grouped by length of ICU stay based the CDC's National Hospital Discharge Survey 2000-2010: https://www.cdc.gov/nchs/data/databriefs/db118.htm#x2013;2010%3C/a%3E%20

In [9]:
def grouplos(los):
    '''
    inputs:
        -los: patient length of stay in ICU
    outputs: 
        Group based on patient length of stay
    '''
    if los >= 10:
        return '10 or more days'
    elif los >= 8:
        return '8-9 days'
    elif los >= 6:
        return '6-7 days'
    elif los >= 4:
        return '4-5 days'
    elif los >= 2:
        return '2-3 days'
    else:
        return '1 day'

dfmimic['los'] = dfmimic.apply(lambda row: grouplos(row['los']), axis=1)
print dfmimic['los'].value_counts()
    

1 day              23078
2-3 days           10733
10 or more days     5171
4-5 days            4039
6-7 days            2127
8-9 days            1328
Name: los, dtype: int64


Now let us create a dataframe with the feature we will want to train our model on

In [10]:
dffeatures = dfmimic[['gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'religion', 'expire_flag', 'los', 'age']]
expire = dfmimic['expire_flag']
print dffeatures.head()

  gender               ethnicity marital_status insurance language  \
3      F                   WHITE       DIVORCED  Medicare     ENGL   
5      F  BLACK/AFRICAN AMERICAN         SINGLE  Self Pay     HAIT   
6      M   UNKNOWN/NOT SPECIFIED           None   Private     None   
8      M                   WHITE         SINGLE   Private     None   
9      F                   WHITE        WIDOWED  Medicare     None   

        religion  expire_flag              los             age  
3       CATHOLIC            0         2-3 days     75-84 years  
5  NOT SPECIFIED            1  10 or more days  Under 45 years  
6          OTHER            0            1 day  Under 45 years  
8   UNOBTAINABLE            0         2-3 days     45-64 years  
9       CATHOLIC            0            1 day     75-84 years  


#### Visualizing the data

Visualize the expiration counts based on the features

In [ ]:
#!/usr/bin/env python
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dfmimic

mu, sigma = 100, 15
x = mu + sigma*np.random.randn(10000)

# the histogram of the data
n, bins, patches = plt.hist(x, 50, normed=1, facecolor='green', alpha=0.75)

plt.xlabel('Smarts')
plt.ylabel('Probability')
plt.title(r'$\mathrm{Histogram\ of\ IQ:}\ \mu=100,\ \sigma=15$')
plt.axis([40, 160, 0, 0.03])
plt.grid(True)

plt.show()

### Part 2: Building the Benchmark Model
First we will start by building a benchmark model using only patient age and length of stay.  We will use only these two features as higher patient age and length of stay are consistently shown to have higher mortality rates.

In [11]:
#Select raw features
benchmark_features_raw = dfmimic[['age', 'los']]
#Select target (patient death)
expiry = dfmimic['expire_flag']

#Use one hot encoding to convert age and LOS groups to dummy variables
benchmark_features = pd.get_dummies(benchmark_features_raw)
print list(benchmark_features.columns)


['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days']


Use sklearn to split the data into test and train data.

In [12]:
from sklearn.model_selection import train_test_split

bench_train, bench_test, exp_train, exp_test = train_test_split(benchmark_features, expiry, random_state=42)

#Check for correct splitting
print "Training set has {} samples.".format(bench_train.shape[0])
print "Testing set has {} samples.".format(bench_test.shape[0])

Training set has 34857 samples.
Testing set has 11619 samples.


Create a training and predicting function to test different machine learning algorithms

In [13]:
def train_results(learner, X_train, y_train, X_test, y_test):
    '''
    inputs:
        -learner: type of learning algorithm to be compared
        -X_train: features training set
        -y_train: expiry training set
        -X_test: features test set
        -y_test: expiry training set
    '''
    
    results = {}
    
    #Fit the learner to the training data
    learner = learner.fit(X_train, y_train)
        
    #Get the predictions on the test set,
    predictions_test = learner.predict(X_test)
        
    #Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test, predictions_test)
        
    #Compute AUC score on the test set
    results['auc_test'] = roc_auc_score(y_test, predictions_test)
        
    # Return the results
    return results

Use GridSearchCV to optimize classifier models.

In [14]:
#Import classifiers to be optimized
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
import random

#Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

#Initialize the classifier
def optimize_classifier(clf, parameters, features, expiry):

    #Split data
    X_train, X_test, y_train, y_test = train_test_split(features, expiry, random_state=random.randint(0,100))

    #Make an auc_score scoring object
    scorer = make_scorer(roc_auc_score)

    #Perform grid search on the classifier using 'scorer' as the scoring method
    grid_obj = GridSearchCV(clf, parameters, scoring=scorer)

    #Fit the grid search object to the training data and find the optimal parameters
    grid_fit = grid_obj.fit(X_train, y_train)

    #Get the estimator
    best_clf = grid_fit.best_estimator_
    print best_clf.get_params()

    # Make predictions using the unoptimized and model
    predictions = (clf.fit(X_train, y_train)).predict(X_test)
    best_predictions = best_clf.predict(X_test)

    # Report the before-and-afterscores
    print "Unoptimized model\n------"
    print "Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions))
    print "AUC score on testing data: {:.4f}".format(roc_auc_score(y_test, predictions))
    print "\nOptimized Model\n------"
    print "Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions))
    print "Final AUC score on the testing data: {:.4f}".format(roc_auc_score(y_test, best_predictions))

#Optimize DecisionTreeClassifier
clf_DTC = tree.DecisionTreeClassifier(random_state = 42)
parameters = {'min_samples_split' : [2,5,8,10], 'max_depth' : [None, 2, 4, 6, 10],
              'min_samples_leaf' : [1,2,3,4]}

optimize_classifier(clf_DTC, parameters, benchmark_features, expiry)

{'presort': False, 'splitter': 'best', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': 42, 'min_impurity_split': 1e-07, 'max_features': None, 'max_depth': 2, 'class_weight': None}
Unoptimized model
------
Accuracy score on testing data: 0.7313
AUC score on testing data: 0.6862

Optimized Model
------
Final accuracy score on the testing data: 0.6990
Final AUC score on the testing data: 0.6975


In [15]:
#Optimize AdaBoostClassifier
clf_ABC = AdaBoostClassifier(random_state = 42)
parameters = {'learning_rate' : [1., 0.9, 0.8, 0.7, 0.6, 0.5], 'n_estimators' : [50, 75, 100]}

optimize_classifier(clf_ABC, parameters, benchmark_features, expiry)

{'n_estimators': 50, 'base_estimator': None, 'random_state': 42, 'learning_rate': 1.0, 'algorithm': 'SAMME.R'}
Unoptimized model
------
Accuracy score on testing data: 0.7329
AUC score on testing data: 0.6933

Optimized Model
------
Final accuracy score on the testing data: 0.7329
Final AUC score on the testing data: 0.6933


In [16]:
#Optimize RandomForestClassifier
clf_RFC = RandomForestClassifier(random_state =42)
parameters = {'max_depth' : [None, 2, 4, 6, 10],}

optimize_classifier(clf_RFC, parameters, benchmark_features, expiry)

{'warm_start': False, 'oob_score': False, 'n_jobs': 1, 'verbose': 0, 'max_leaf_nodes': None, 'bootstrap': True, 'min_samples_leaf': 1, 'n_estimators': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': 42, 'min_impurity_split': 1e-07, 'max_features': 'auto', 'max_depth': None, 'class_weight': None}
Unoptimized model
------
Accuracy score on testing data: 0.7410
AUC score on testing data: 0.6925

Optimized Model
------
Final accuracy score on the testing data: 0.7410
Final AUC score on the testing data: 0.6925


Create models using features that are included in each ICU admission (gender, insurance).  Add these features to age and LOS.

In [17]:
def classifier_for_features(df, features, trials):
    '''
    df - The Pandas DataFrame with the data for training and testing
    features - A list of column features the classifier will use for training
    trials - An integer for the number of random trials the best classifier function will run
    '''
    
    #Preprocess feature data
    input_features_raw = df[features]
    input_features = pd.get_dummies(input_features_raw)
    
    #Get target variable
    expiry = df['expire_flag']
    
    #Assign dictionary name to features
    features_key = ' '.join(features)
    if df.name not in best_classifiers:
        best_classifiers[df.name] = {features_key: []}
    else:
        if features_key not in best_classifiers[df.name]:
            best_classifiers[df.name][features_key] = []
    
    #Add features to list for future data collection
    if features not in previous_features:
        previous_features.append(features)
        
    
    #Run best_classifier function
    best_classifiers[df.name][features_key] = \
    best_classifiers[df.name][features_key] + best_classifier(input_features, expiry, trials, clf_name_list)
    print best_classifiers
    print previous_features
    
    

Evaluate classifiers:

In [18]:
#Import useful modules

import mimic_visuals as vs
from time import time

#Set up results dictionary for classifiers
clf_A = GaussianNB()
clf_B = tree.DecisionTreeClassifier(random_state=42)
clf_C = RandomForestClassifier(random_state =42)
clf_D = AdaBoostClassifier(random_state = 42)
#clf_E = SVC(random_state = 42)

clf_name_list = []
for clf in [clf_A, clf_B, clf_C, clf_D]:
    clf_name = clf.__class__.__name__
    clf_name_list.append(clf_name)

best_classifiers = {} #Dictionary to store best classifiers and AUC scores
previous_features = []  #List to store features that have been analyzed
    
def best_classifier(features, expiry, trials, classifiers):
    '''
        -features: A dataframe containing the features for training
        -expiry: Target variable, binary patient death or survival
        -trials:  An integer denoting the number of random iterations
        -classifiers:  A list of supervised learning classifiers
    '''
    
    #Run through multiple times using random states
    results = {}
    for clf_name in classifiers:
        results[clf_name] = {}
    for trial in range(trials):
        state = random.randint(0,429496729)
        #print "Trial {}: Random State is {}".format(trial+1, state)
    
        #Create test and train samples
        X_train, X_test, y_train, y_test = train_test_split(features, expiry,
                                                                   random_state=random.randint(0,429496729))

        #  Initialize the models
        clf_A = GaussianNB()
        clf_B = tree.DecisionTreeClassifier(random_state=state)
        clf_C = RandomForestClassifier(random_state =state)
        clf_D = AdaBoostClassifier(random_state = state)
        #clf_E = SVC(random_state = state)       

        # Collect results on the learners
        for clf in [clf_A, clf_B, clf_C, clf_D]:
            clf_name = clf.__class__.__name__
            results[clf_name][trial] = \
            train_results(clf, X_train, y_train, X_test, y_test)

    #Find average AUC scores
    average_scores = {'accuracy': {}, 'auc': {}}
    #print results
    for clf in results:
        auc_scores = 0
        accuracy_scores = 0
        for trial in results[clf]:
            auc_scores+= results[clf][trial]['auc_test']
            accuracy_scores += results[clf][trial]['acc_test']
        average_scores['auc'][clf] = auc_scores/trials
        average_scores['accuracy'][clf] = accuracy_scores/trials   

    # Print final AUC scores
    #for clf in results:
    #    print "{} has an average AUC test score of {:.6f}".format(clf, average_scores['auc'][clf])
    #    print "{} has an average accuracy test score of {:.6f}".format(clf, average_scores['accuracy'][clf])
    #Print classifier with max AUC score
    max_auc_clf = max(average_scores['auc'].iterkeys(), key=(lambda key: average_scores['auc'][key]))
    print "{} has highest AUC test score of {:.6f}".format(max_auc_clf, average_scores['auc'][max_auc_clf])
    max_accuracy_clf = max(average_scores['accuracy'].iterkeys(), key=(lambda key: average_scores['accuracy'][key]))
    print "{} has highest accuracy score of {:.6f}".format(max_accuracy_clf, average_scores['accuracy'][max_accuracy_clf])
    #Return highest scores to be stored in dictionary
    return [max_auc_clf, average_scores['auc'][max_auc_clf]]
      
classifier_for_features(dfmimic, ['age', 'los'], 10)

GaussianNB has highest AUC test score of 0.693113
DecisionTreeClassifier has highest accuracy score of 0.733497
{'dfmimic': {'age los': ['GaussianNB', 0.69311261002135482]}}
[['age', 'los']]


From the benchmark model testing, we find that the GaussianNB has the highest AUC score with 0.6980.  With this as the chosen classification model, we can see whether we can improve this score. 

### Part 3: Building a model including gender and type of insurance


Create a function that will allow for quicker comparisons among different groups of features

In [19]:
#Select features
features_in_all_admissions = ['age', 'los', 'gender', 'insurance']
#Find best classifier for features included in all ICU admissions
classifier_for_features(dfmimic, features_in_all_admissions, 10)

GaussianNB has highest AUC test score of 0.712478
AdaBoostClassifier has highest accuracy score of 0.735046
{'dfmimic': {'age los': ['GaussianNB', 0.69311261002135482], 'age los gender insurance': ['GaussianNB', 0.71247805218304128]}}
[['age', 'los'], ['age', 'los', 'gender', 'insurance']]


We can see that adding 'gender' and 'insurance' features has improved the AUC score of the model from 0.6990 to 0.7119.  Let us further look into this to see whether adding one feature or the other changes the AUC score.

In [20]:
#Select benchmark features plus gender
benchmark_features_gender = ['age', 'los', 'gender']
#Find best classifier for features included in all ICU admissions
classifier_for_features(dfmimic, benchmark_features_gender, 10)

GaussianNB has highest AUC test score of 0.698533
AdaBoostClassifier has highest accuracy score of 0.733781
{'dfmimic': {'age los gender': ['GaussianNB', 0.69853290323476736], 'age los': ['GaussianNB', 0.69311261002135482], 'age los gender insurance': ['GaussianNB', 0.71247805218304128]}}
[['age', 'los'], ['age', 'los', 'gender', 'insurance'], ['age', 'los', 'gender']]


Best AUC score for 'gender' is GaussianNB classifier with AUC score of 0.6964

In [21]:
#Select benchmark features plus insurance
benchmark_features_insurance = ['age', 'los', 'insurance']
#Find best classifier for features included in all ICU admissions
classifier_for_features(dfmimic, benchmark_features_insurance, 10)

GaussianNB has highest AUC test score of 0.713758
AdaBoostClassifier has highest accuracy score of 0.734237
{'dfmimic': {'age los gender': ['GaussianNB', 0.69853290323476736], 'age los': ['GaussianNB', 0.69311261002135482], 'age los insurance': ['GaussianNB', 0.71375759895520674], 'age los gender insurance': ['GaussianNB', 0.71247805218304128]}}
[['age', 'los'], ['age', 'los', 'gender', 'insurance'], ['age', 'los', 'gender'], ['age', 'los', 'insurance']]


Best AUC score for 'insurance' is GaussianNB with 0.7128.  This is slightly better than both 'insurance' and 'gender' features added.  

### Part 4: Adding optional features to the model

Let us now look at the data with the optional features ('ethnicity', 'marital_status', 'religion', and 'language' added.  For these optional features we will be using the cleaned groups dataframe with rows containing 'NaN', 'UNKOWN', and 'UNABLE TO OBTAIN' values removed.

First let's create a function to allow us to compare how classifiers using the baseline features with the cleaned up DataFrame compare to classifiers using the optional features

In [22]:
def baseline_features(df, previous_features, trials):
    '''
        -df: The cleaned up DataFrame
        -previous_features: A list containing lists of feature names
        -trials: Number of randomized trials to run to find average AUC score
    '''
    
    for i in range(0,4): #Baseline features stored in indices 0-3
        classifier_for_features(df, previous_features[i], trials)

For ethnicity we will only look at the four ethnicities (white, black, hispanic, asian) contained within this study: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2569791/.  This will help reduce the amount of features in the model without removing too many data points.

In [23]:
#Ethnicity
#Clean up ethnicity DataFrame
df_ethnicity = dfmimic[['age', 'gender', 'los', 'insurance', 'ethnicity', 'expire_flag', 'subject_id']]
jnma_ethnicity = ['WHITE', 'BLACK/AFRICAN AMERICAN', 'HISPANIC OR LATINO', 'ASIAN']
df_ethnicity = df_ethnicity[df_ethnicity['ethnicity'].isin(jnma_ethnicity)]
#Name df_ethnicity
df_ethnicity.name = 'df_ethnicity'

#Get baseline features data
baseline_features(df_ethnicity, previous_features, 10)    

#Use 'ethnicity' with 'age', 'los', 'gender', and 'insurance'
all_features_ethnicity = ['age', 'los', 'insurance', 'gender', 'ethnicity']

classifier_for_features(df_ethnicity, all_features_ethnicity, 10)

GaussianNB has highest AUC test score of 0.700394
DecisionTreeClassifier has highest accuracy score of 0.732585
{'df_ethnicity': {'age los': ['GaussianNB', 0.70039398256631213]}, 'dfmimic': {'age los gender': ['GaussianNB', 0.69853290323476736], 'age los': ['GaussianNB', 0.69311261002135482], 'age los insurance': ['GaussianNB', 0.71375759895520674], 'age los gender insurance': ['GaussianNB', 0.71247805218304128]}}
[['age', 'los'], ['age', 'los', 'gender', 'insurance'], ['age', 'los', 'gender'], ['age', 'los', 'insurance']]
GaussianNB has highest AUC test score of 0.712921
AdaBoostClassifier has highest accuracy score of 0.734956
{'df_ethnicity': {'age los': ['GaussianNB', 0.70039398256631213], 'age los gender insurance': ['GaussianNB', 0.71292094617912949]}, 'dfmimic': {'age los gender': ['GaussianNB', 0.69853290323476736], 'age los': ['GaussianNB', 0.69311261002135482], 'age los insurance': ['GaussianNB', 0.71375759895520674], 'age los gender insurance': ['GaussianNB', 0.7124780521830

Best AUC score for 'ethnicity' is GaussianNB with 0.7025  

In [24]:
#Marital Status
#Clean up marital status DataFrame
df_marital_status = dfmimic[['age', 'gender', 'los', 'insurance', 'marital_status', 'expire_flag', 'subject_id']]
df_marital_status = df_marital_status.dropna()
df_marital_status = df_marital_status[df_marital_status.marital_status != 'UNKNOWN (DEFAULT)']
#Name df_marital_status
df_marital_status.name = 'df_marital_status'

#Get baseline features data
baseline_features(df_marital_status, previous_features, 10)   

#Use 'marital_status' with 'age', 'los', 'gender', and 'insurance'
all_features_marital_status = ['age', 'los', 'insurance', 'gender', 'marital_status']

classifier_for_features(df_marital_status, all_features_marital_status, 10)

DecisionTreeClassifier has highest AUC test score of 0.658095
DecisionTreeClassifier has highest accuracy score of 0.682830
{'df_ethnicity': {'age los gender': ['GaussianNB', 0.6981347594656413], 'age los insurance gender ethnicity': ['GaussianNB', 0.70360573617704403], 'age los': ['GaussianNB', 0.70039398256631213], 'age los insurance': ['GaussianNB', 0.71233467028557929], 'age los gender insurance': ['GaussianNB', 0.71292094617912949]}, 'df_marital_status': {'age los': ['DecisionTreeClassifier', 0.65809535308067124]}, 'dfmimic': {'age los gender': ['GaussianNB', 0.69853290323476736], 'age los': ['GaussianNB', 0.69311261002135482], 'age los insurance': ['GaussianNB', 0.71375759895520674], 'age los gender insurance': ['GaussianNB', 0.71247805218304128]}}
[['age', 'los'], ['age', 'los', 'gender', 'insurance'], ['age', 'los', 'gender'], ['age', 'los', 'insurance'], ['age', 'los', 'insurance', 'gender', 'ethnicity']]
GaussianNB has highest AUC test score of 0.664228
AdaBoostClassifier has

Best AUC score for 'marital_status' is AdaBoostClassifier with 0.6585

In [25]:
#Language
#Clean up language DataFrame
df_language = dfmimic[['age', 'gender', 'los', 'insurance', 'language', 'expire_flag', 'subject_id']]
df_language = df_language.dropna()
#Adjust language for to use only top 5 spoken languages
print df_language['language'].value_counts()
top_5 = ['ENGL', 'SPAN', 'RUSS', 'PTUN', 'CANT']
df_language_5 = df_language[df_language['language'].isin(top_5)]
#Name df_language
df_language.name = 'df_language'

#Get baseline features data
baseline_features(df_language, previous_features, 10)   

#Use 'language' with 'age', 'los', 'gender', and 'insurance'
all_features_language = ['age', 'los', 'insurance', 'gender', 'language']

classifier_for_features(df_language, all_features_language, 10)

ENGL    21733
SPAN      800
RUSS      531
PTUN      522
CANT      320
PORT      274
CAPE      201
MAND      135
HAIT      106
ITAL       80
VIET       69
GREE       55
ARAB       28
PERS       24
POLI       23
HIND       23
CAMB       22
AMER       22
KORE       20
ALBA       15
FREN       13
THAI        9
*ARM        8
*BEN        7
LAOT        6
ETHI        6
SOMA        6
*URD        4
*GUJ        4
*CDI        4
        ...  
*HUN        2
** T        1
*YOR        1
*TAM        1
*LIT        1
*DEA        1
BENG        1
*PER        1
*TOY        1
*MOR        1
*KHM        1
*CAN        1
*BOS        1
*PHI        1
*FIL        1
*FUL        1
*NEP        1
* BE        1
*SPA        1
*ROM        1
*ARA        1
GERM        1
**SH        1
*RUS        1
*CRE        1
SERB        1
*MAN        1
*PUN        1
* FU        1
*FAR        1
Name: language, dtype: int64
GaussianNB has highest AUC test score of 0.635913
AdaBoostClassifier has highest accuracy score of 0.713224
{'df_ethn

In [26]:
#Religion
#Clean up religion DataFrame
df_religion = dfmimic[['age', 'gender', 'los', 'insurance', 'religion', 'expire_flag', 'subject_id']]
df_religion = df_religion.dropna()
#Keep religions that have value counts >= 10
religion_counts = df_religion['religion'].value_counts()
df_religion = df_religion[df_religion['religion'].isin(religion_counts[religion_counts >= 10].index)]
df_religion = df_religion[df_religion.religion != 'UNOBTAINABLE']
#Name df_religion
df_religion.name = 'df_religion'

#Get baseline features data
baseline_features(df_religion, previous_features, 10)   

#Use 'religion' with 'age', 'los', 'gender', and 'insurance'
all_features_religion = ['age', 'los', 'insurance', 'gender', 'religion']

classifier_for_features(df_religion, all_features_religion, 10)

GaussianNB has highest AUC test score of 0.688989
AdaBoostClassifier has highest accuracy score of 0.719944
{'df_ethnicity': {'age los gender': ['GaussianNB', 0.6981347594656413], 'age los insurance gender ethnicity': ['GaussianNB', 0.70360573617704403], 'age los': ['GaussianNB', 0.70039398256631213], 'age los insurance': ['GaussianNB', 0.71233467028557929], 'age los gender insurance': ['GaussianNB', 0.71292094617912949]}, 'df_language': {'age los gender': ['GaussianNB', 0.62745489212729932], 'age los insurance gender language': ['AdaBoostClassifier', 0.61403931196770689], 'age los': ['GaussianNB', 0.63591278647581895], 'age los insurance': ['GaussianNB', 0.6563752152360961], 'age los gender insurance': ['GaussianNB', 0.6492045397647348]}, 'df_marital_status': {'age los gender': ['GaussianNB', 0.65575191361767082], 'age los insurance gender marital_status': ['GaussianNB', 0.66233010169165329], 'age los': ['DecisionTreeClassifier', 0.65809535308067124], 'age los insurance': ['RandomFore

Best score is AdaBoostClassifier with AUC score of 0.6763

### Part 5: Merging optional features
#### 2 optional features

In [40]:
#Merge ethnicity and marital status
df_ems = df_ethnicity.merge(df_marital_status, how='inner',
                                           on = ['subject_id', 'age', 'gender', 'los', 'insurance', 'expire_flag'])
df_ems.name = 'df_ems'
print len(df_ethnicity.index)
print len(df_marital_status.index)
print len(df_ems.index)
print list(df_ems.columns)

#Get baseline features data
baseline_features(df_ems, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'ethnicity' 'marital_status and 'gender' features
ems_features = ['age', 'los', 'insurance', 'gender', 'ethnicity', 'marital_status']

classifier_for_features(df_ems, ems_features, 10)

38298
36412
30615
['age', 'gender', 'los', 'insurance', 'ethnicity', 'expire_flag', 'subject_id', 'marital_status']
DecisionTreeClassifier has highest AUC test score of 0.655269
DecisionTreeClassifier has highest accuracy score of 0.679370
{'df_ems': {'age los': ['DecisionTreeClassifier', 0.65526927292470594]}, 'df_language': {'age los gender': ['GaussianNB', 0.62745489212729932], 'age los insurance gender language': ['AdaBoostClassifier', 0.61403931196770689], 'age los': ['GaussianNB', 0.63591278647581895], 'age los insurance': ['GaussianNB', 0.6563752152360961], 'age los gender insurance': ['GaussianNB', 0.6492045397647348]}, 'df_marital_status': {'age los gender': ['GaussianNB', 0.65575191361767082], 'age los insurance gender marital_status': ['GaussianNB', 0.66233010169165329], 'age los': ['DecisionTreeClassifier', 0.65809535308067124], 'age los insurance': ['RandomForestClassifier', 0.66332018372879131], 'age los gender insurance': ['GaussianNB', 0.66422800929601422]}, 'df_ethnici

In [41]:
#Merge ethnicity and religion
df_er = df_ethnicity.merge(df_religion, how='inner',
                                           on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'subject_id'])
df_er.name = 'df_er'
print len(df_er.index)

#Get baseline features data
baseline_features(df_er, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'ethnicity' 'religion' and 'gender' features
er_features = ['age', 'los', 'insurance', 'gender', 'ethnicity', 'religion']

classifier_for_features(df_er, er_features, 10)

32956
GaussianNB has highest AUC test score of 0.686305
AdaBoostClassifier has highest accuracy score of 0.717490
{'df_er': {'age los': ['GaussianNB', 0.68630457774518216]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['GaussianNB', 0.66083920623453296], 'age los insurance gender ethnicity marital_status': ['AdaBoostClassifier', 0.65398278676214694]}, 'df_language': {'age los gender': ['GaussianNB', 0.62745489212729932], 'age los insurance gender language': ['AdaBoostClassifier', 0.61403931196770689], 'age los': ['GaussianNB', 0.63591278647581895], 'age los insurance': ['GaussianNB', 0.6563752152360961], 'age los gender insurance': ['GaussianNB', 0.6492045397647348]}, 'df_marital_status': {'age los gender': ['GaussianNB', 0.65575191361767082], 'age los insurance gender marital_status': ['GaussianNB', 0.662330

In [42]:
#Merge ethnicity and language
df_el = df_ethnicity.merge(df_language, how='inner',
                                           on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'subject_id'])
df_el.name = 'df_el'
print len(df_el.index)

#Get baseline features data
baseline_features(df_el, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'ethnicity' 'language' and 'gender' features
el_features = ['age', 'los', 'insurance', 'gender', 'ethnicity', 'language']

classifier_for_features(df_el, el_features, 10)

21358
GaussianNB has highest AUC test score of 0.634833
AdaBoostClassifier has highest accuracy score of 0.707734
{'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['GaussianNB', 0.66083920623453296], 'age los insurance gender ethnicity marital_status': ['AdaBoostClassifier', 0.65398278676214694]}, 'df_language': {'age los gender': ['GaussianNB', 0.62745489212729932], 'age los insurance gender language': ['AdaBoostClassifier', 0.61403931196770689], 'age los': ['GaussianNB', 0.63591278647581895], 'age 

In [43]:
#Merge marital_status and religion
df_msr = df_marital_status.merge(df_religion, how='inner',
                                           on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'subject_id'])
df_msr.name = 'df_msr'
print len(df_msr.index)

#Get baseline features data
baseline_features(df_msr, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'religion' 'marital_status' and 'gender' features
msr_features = ['age', 'los', 'insurance', 'gender', 'marital_status', 'religion']

classifier_for_features(df_msr, msr_features, 10)

32489
RandomForestClassifier has highest AUC test score of 0.642205
AdaBoostClassifier has highest accuracy score of 0.680007
{'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['GaussianNB', 0.66083920623453296], 'age los insurance gender ethnicity marital_status': ['AdaBoostClassifier', 0.65398278676214694]}, 'df_language': {'age los gender': ['GaussianNB', 0.62745489212729932], 'age los insurance gender language': ['AdaBoostClassifier', 0.61403931196770689], 'age los': ['GaussianNB', 0.6359127864758

In [45]:
#Merge marital_status and language
df_msl = df_marital_status.merge(df_language, how='inner',
                                           on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'subject_id'])
df_msl.name = 'df_msl'
print len(df_msl.index)



#Get baseline features data
baseline_features(df_msl, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'language' 'marital_status' and 'gender' features
msl_features = ['age', 'los', 'insurance', 'gender', 'marital_status', 'language']

classifier_for_features(df_msl, msl_features, 10)

23859
GaussianNB has highest AUC test score of 0.623955
AdaBoostClassifier has highest accuracy score of 0.711769
{'df_msl': {'age los': ['GaussianNB', 0.62395481882304282]}, 'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['GaussianNB', 0.66083920623453296], 'age los insurance gender ethnicity marital_status': ['AdaBoostClassifier', 0.65398278676214694]}, 'df_language': {'age los gender': ['GaussianNB', 0.62745489212729932], 'age los insurance gender language': ['AdaBoostClassifier', 0.6140393119677

In [47]:
#Merge religion and language
df_rl = df_religion.merge(df_language, how='inner',
                                           on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'subject_id'])
df_rl.name = 'df_rl'
print len(df_msr.index)

#Get baseline features data
baseline_features(df_rl, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'language' 'religion' and 'gender' features
rl_features = ['age', 'los', 'insurance', 'gender', 'religion', 'language']

classifier_for_features(df_rl, rl_features, 10)

32489
GaussianNB has highest AUC test score of 0.623808
AdaBoostClassifier has highest accuracy score of 0.713751
{'df_msl': {'age los gender': ['GaussianNB', 0.62145495374603998], 'age los': ['GaussianNB', 0.62395481882304282], 'age los insurance': ['GaussianNB', 0.6559854679987579], 'age los gender insurance': ['GaussianNB', 0.65404314060948221], 'age los insurance gender marital_status language': ['AdaBoostClassifier', 0.61169805375786357]}, 'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['Gaussi

#### Merge 3 optional features

In [49]:
#Merge ethnicity, religion, and marital status
df_erms = df_ems.merge(df_er, how='inner',
                        on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'ethnicity', 'subject_id'])
df_erms.name = 'df_erms'
print len(df_erms.index)

#Get baseline features data
baseline_features(df_erms, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'ethnicity' 
# 'religion' 'marital_status' and 'gender' features
erms_features = ['age', 'los', 'insurance', 'gender', 'religion', 'ethnicity', 'marital_status']

classifier_for_features(df_erms, erms_features, 10)

27970
GaussianNB has highest AUC test score of 0.640517
AdaBoostClassifier has highest accuracy score of 0.678350
{'df_msl': {'age los gender': ['GaussianNB', 0.62145495374603998], 'age los': ['GaussianNB', 0.62395481882304282], 'age los insurance': ['GaussianNB', 0.6559854679987579], 'age los gender insurance': ['GaussianNB', 0.65404314060948221], 'age los insurance gender marital_status language': ['AdaBoostClassifier', 0.61169805375786357]}, 'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['Gaussi

In [50]:
#Merge ethnicity, language, and marital status
df_elms = df_ems.merge(df_el, how='inner',
                        on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'ethnicity', 'subject_id'])
df_elms.name = 'df_elms'
print len(df_elms.index)

#Get baseline features data
baseline_features(df_elms, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'language' 
# 'ethnicity' 'marital_status' and 'gender' features
elms_features = ['age', 'los', 'insurance', 'gender', 'ethnicity', 'language', 'marital_status']

classifier_for_features(df_elms, elms_features, 10)

20818
GaussianNB has highest AUC test score of 0.628252
AdaBoostClassifier has highest accuracy score of 0.705744
{'df_msl': {'age los gender': ['GaussianNB', 0.62145495374603998], 'age los': ['GaussianNB', 0.62395481882304282], 'age los insurance': ['GaussianNB', 0.6559854679987579], 'age los gender insurance': ['GaussianNB', 0.65404314060948221], 'age los insurance gender marital_status language': ['AdaBoostClassifier', 0.61169805375786357]}, 'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier', 0.65896081716838117], 'age los gender insurance': ['Gaussi

In [51]:
#Merge religion, language, and marital status
df_msrl = df_msl.merge(df_msr, how='inner',
                        on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'marital_status', 'subject_id'])
df_msrl.name = 'df_msrl'
print len(df_msrl.index)

#Get baseline features data
baseline_features(df_msrl, previous_features, 10)   

#Find best classifier with 'age' 'los' 'insurance' 'language' 
# 'religion' 'marital_status' and 'gender' features
msrl_features = ['age', 'los', 'insurance', 'gender', 'religion', 'language', 'marital_status']

classifier_for_features(df_msrl, msrl_features, 10)

22176
GaussianNB has highest AUC test score of 0.626313
AdaBoostClassifier has highest accuracy score of 0.716144
{'df_msrl': {'age los': ['GaussianNB', 0.62631286877437364]}, 'df_msl': {'age los gender': ['GaussianNB', 0.62145495374603998], 'age los': ['GaussianNB', 0.62395481882304282], 'age los insurance': ['GaussianNB', 0.6559854679987579], 'age los gender insurance': ['GaussianNB', 0.65404314060948221], 'age los insurance gender marital_status language': ['AdaBoostClassifier', 0.61169805375786357]}, 'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB', 0.6969062866496214], 'age los insurance gender ethnicity religion': ['GaussianNB', 0.67810511613938407]}, 'df_ems': {'age los gender': ['GaussianNB', 0.65187170219060875], 'age los': ['DecisionTreeClassifier', 0.65526927292470594], 'age los insurance': ['RandomForestClassifier

#### All features combined

In [52]:
#Merge religion, language, ethnicity, and marital status
df_emsrl = df_msrl.merge(df_elms, how='inner',
                        on = ['age', 'gender', 'los', 'insurance', 'expire_flag', 'marital_status', 'language', 'subject_id'])
df_emsrl.name = 'df_emsrl'

#Get AUC scores for all previous feature combinations with fully merged data frame
for feature_set in previous_features:
    classifier_for_features(df_emsrl, feature_set, 10)

#Get AUC score using all features
# 'age', 'los', 'insurance', 'gender', 'religion', 'language', 'marital_status', and 'ethnicity'
emsrl_features = ['age', 'los', 'insurance', 'gender', 'religion', 'language', 'marital_status']
classifier_for_features(df_emsrl, emsrl_features, 10)

GaussianNB has highest AUC test score of 0.621242
AdaBoostClassifier has highest accuracy score of 0.708196
{'df_msrl': {'age los gender': ['GaussianNB', 0.62364048744119616], 'age los': ['GaussianNB', 0.62631286877437364], 'age los insurance': ['GaussianNB', 0.65173727032337159], 'age los gender insurance': ['GaussianNB', 0.64878731322527372], 'age los insurance gender religion language marital_status': ['AdaBoostClassifier', 0.60510148010431197]}, 'df_msl': {'age los gender': ['GaussianNB', 0.62145495374603998], 'age los': ['GaussianNB', 0.62395481882304282], 'age los insurance': ['GaussianNB', 0.6559854679987579], 'age los gender insurance': ['GaussianNB', 0.65404314060948221], 'age los insurance gender marital_status language': ['AdaBoostClassifier', 0.61169805375786357]}, 'df_er': {'age los gender': ['GaussianNB', 0.68446185148811478], 'age los': ['GaussianNB', 0.68630457774518216], 'age los insurance': ['GaussianNB', 0.6957159393066028], 'age los gender insurance': ['GaussianNB',

### Part 6: Comparing Best Classifiers

In [56]:
#Find best classifier for each DataFrame
for df in best_classifiers:
    best_features = max(best_classifiers[df].iterkeys(), key=(lambda key: best_classifiers[df][key][1]))
    best_learner, learner_auc_score = best_classifiers[df][best_features][0], best_classifiers[df][best_features][1]
    print "{} : {} ".format(best_features, df)
    print "{} : AUC score {:.6f}".format(best_learner, learner_auc_score)

age los insurance : df_msrl 
GaussianNB : AUC score 0.651737
age los insurance : df_msl 
GaussianNB : AUC score 0.655985
age los gender insurance : df_er 
GaussianNB : AUC score 0.696906
age los gender insurance : df_ems 
GaussianNB : AUC score 0.660839
age los insurance : df_emsrl 
GaussianNB : AUC score 0.647108
age los insurance : df_language 
GaussianNB : AUC score 0.656375
age los gender insurance : df_marital_status 
GaussianNB : AUC score 0.664228
age los gender insurance : df_ethnicity 
GaussianNB : AUC score 0.712921
age los gender insurance : df_elms 
GaussianNB : AUC score 0.648945
age los gender insurance : df_erms 
GaussianNB : AUC score 0.655776
age los insurance : df_religion 
GaussianNB : AUC score 0.699209
age los gender insurance : df_rl 
GaussianNB : AUC score 0.653928
age los insurance : df_el 
GaussianNB : AUC score 0.653921
age los insurance : dfmimic 
GaussianNB : AUC score 0.713758
age los gender insurance : df_msr 
GaussianNB : AUC score 0.661010
